In [ ]:
import datetime as dt
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from tqdm import tqdm

# ***Copernicus Global Land Service Resampler***

## Purpose

This notebook shows how to resample Copernicus Global Land Service vegetation-related products (i.e. NDVI, FAPAR...), based on PROBA-V observations, from 333m resolution to 1km using R-based packages and functions.

It is intended for users who would like to continue, temporarily, their 1km time series, in near real-time, before switching to the new 333m baseline resolution.

## Step 1: Data selection or download

Two approaches are preset to manage the data income; the user can define a specific file that is already present locally or, thanks to the downloader, select the specific product and date of interest. Both the approaches the definition of the output folder is needed.
In this folder will be written the downloaded product and the resampled file.

For more details on the products, please see the description and Product User Manuals documentation at https://land.copernicus.eu/global/products/

Define the output folder. If empty the same folder of the notebook will be used. 

In [ ]:
folder = ''

### Option 1: Product is localy available

If the product is already available locally, the path must be defined as a string.

<span style="color:red">**NOTE!:Input file must be in netCDF (.nc) format.If needed a tiff file can be read but a modification according to the xarray documentation must be done.**</span>   

In [ ]:
path = ''

### Option 2: Automatic procedure to download data from the Copernicus Global Land repository

To get access to the Copernicus Global Land repository user credential must be provided.  

Credential can be obtained here https://land.copernicus.vgt.vito.be/PDF/portal/Application.html#Home throught the Register form (on the upper right part of the page)

In [ ]:
user = ''
psw = ''

Select the product and date of interest. Wait until the download is finish.
The selected date will be adjusted to the closest date available for the selected product. 

In [ ]:
url = 'https://land.copernicus.vgt.vito.be/manifest/'

session = requests.Session()
session.auth = (user, psw)

manifest = session.get(url, allow_redirects=True)
products = pd.read_html(manifest.text)[0][2:-1]['Name']
products = products[products.str.contains('300_')].reset_index(drop=True)
print(products)
val = input('Please select the product from the list:')
url = f'{url}{products[int(val)]}'

manifest = session.get(url, allow_redirects=True)
product = pd.read_html(manifest.text)[0][-2:-1]['Name'].values[0]
purl = f'{url}{product}'
r = session.get(purl, stream=True)
rows = r.text.split('\n')
dates = pd.DataFrame()
for line in rows[:-1]:
    r = re.search(r"\d\d\d\d(\/)\d\d(\/)\d\d", line)
    dates = dates.append(pd.DataFrame([line], index=[pd.to_datetime(r[0], format="%Y/%m/%d")]))

val = input('Please insert the date in the format YYYY/MM/DD:')

dates = dates.sort_index()
i = dates.index.searchsorted(dt.datetime.strptime(val, "%Y/%m/%d"))
link = dates.iloc[i][0]
filename = os.path.basename(link)
if folder != '':
    path = sys.path.join(folder, filename)
else:
    path = filename

r = session.get(link, stream=True)

total_size = int(r.headers.get('content-length', 0))
block_size = 1024  # 1 Kibibyte
t = tqdm(total=total_size, unit='iB', unit_scale=True)

with open(path, 'wb') as f:
    for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
t.close()
if total_size != 0 and t.n != total_size:
    print("ERROR, something went wrong")

## Step 2: Data ingestion

Dataset is readed and parameters are adjusted according to the product. No extra effort is needed to define the correct product parameters.  

In [ ]:
ds = xr.open_dataset(path, mask_and_scale=False)

Automatic select of products parametes.

In [ ]:
if 'LAI' in ds.data_vars:
    param = {'product': 'LAI',
             'short_name': 'leaf_area_index',
             'long_name': 'Leaf Area Index Resampled 1 Km',
             'grid_mapping': 'crs',
             'flag_meanings': 'Missing',
             'flag_values': '255',
             'units': '',
             'PHYSICAL_MIN': 0,
             'PHYSICAL_MAX': 7,
             'DIGITAL_MAX': 210,
             'SCALING': 1. / 30,
             'OFFSET': 0}
    da = ds.LAI
elif 'FCOVER' in ds.data_vars:
    param = {'product': 'FCOVER',
             'short_name': 'vegetation_area_fraction',
             'long_name': 'Fraction of green Vegetation Cover Resampled 1 Km',
             'grid_mapping': 'crs',
             'flag_meanings': 'Missing',
             'flag_values': '255',
             'units': '',
             'valid_range': '',
             'PHYSICAL_MIN': 0,
             'PHYSICAL_MAX': 1.,
             'DIGITAL_MAX': 250,
             'SCALING': 1. / 250,
             'OFFSET': 0}
    da = ds.FCOVER
elif 'FAPAR' in ds.data_vars:
    param = {'product': 'FAPAR',
             'short_name': 'Fraction_of_Absorbed_Photosynthetically_Active_Radiation',
             'long_name': 'Fraction of Absorbed Photosynthetically Active Radiation Resampled 1 KM',
             'grid_mapping': 'crs',
             'flag_meanings': 'Missing',
             'flag_values': '255',
             'units': '',
             'valid_range': '',
             'PHYSICAL_MIN': 0,
             'PHYSICAL_MAX': 0.94,
             'DIGITAL_MAX': 235,
             'SCALING': 1. / 250,
             'OFFSET': 0}
    da = ds.FAPAR
elif 'NDVI' in ds.data_vars:
    param = {'product': 'NDVI',
             'short_name': 'Normalized_difference_vegetation_index',
             'long_name': 'Normalized Difference Vegetation Index Resampled 1 Km',
             'grid_mapping': 'crs',
             'flag_meanings': 'Missing cloud snow sea background',
             'flag_values': '[251 252 253 254 255]',
             'units': '',
             'PHYSICAL_MIN': -0.08,
             'PHYSICAL_MAX': 0.92,
             'DIGITAL_MAX': 250,
             'SCALING': 1. / 250,
             'OFFSET': -0.08}
    da = ds.NDVI
elif 'DMP' in ds.data_vars:
    param = {'product': 'DMP',
             'short_name': 'dry_matter_productivity',
             'long_name': 'Dry matter productivity Resampled 1KM',
             'grid_mapping': 'crs',
             'flag_meanings': 'sea',
             'flag_values': '-2',
             'units': 'kg / ha / day',
             'PHYSICAL_MIN': 0,
             'PHYSICAL_MAX': 327.67,
             'DIGITAL_MAX': 32767,
             'SCALING': 1. / 100,
             'OFFSET': 0}
    da = ds.DMP
elif 'GDMP' in ds.data_vars:
    param = {'product': 'GDMP',
             'short_name': 'Gross_dry_matter_productivity',
             'long_name': 'Gross dry matter productivity Resampled 1KM',
             'grid_mapping': 'crs',
             'flag_meanings': 'sea',
             'flag_values': '-2',
             'units': 'kg / hectare / day',
             'PHYSICAL_MIN': 0,
             'PHYSICAL_MAX': 655.34,
             'DIGITAL_MAX': 32767,
             'SCALING': 1. / 50,
             'OFFSET': 0}
    da = ds.GDMP
else:
    sys.exit('GLC product not found please chek')

In [ ]:
_, tail = os.path.split(path)
pos = [pos for pos, char in enumerate(tail) if char == '_'][2]
date = tail[pos + 1: pos + 9]
date_h = dt.datetime.strptime(date, '%Y%m%d').date()

# Step 3: Selection of the Area of Interest

To facilitate users, an Area of Interest (AOI), can be defined in the format [Long_min, Long_Max, Lat_min, Lat_Max]. Values must be expressed in decimal degree.

The extent will be adjusted according to the cell grid of the equivalent 1 Km product. If the AOI does not fit in the new grid, the leftish boundary will be truncated to the closest border of the 1km grid. 

If the AOI is not defined the entire dataset will be processed. As a global 300m data array has 120960 columns and 47040 rows, or close to 5.7bn cells, be aware that this could imply heavy use of computer resources. Xarray can easily manage this thanks to the lazy approach, but if it fails an extra effort is needed to define the chunk sizes manually. 
More info is available here: http://xarray.pydata.org/en/stable/dask.html

AOI definition 

In [ ]:
AOI = [-18.58, 62.95, 51.57, 28.5]

AOI extent controll

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def bnd_box_adj(my_ext):
    lat_1k = np.arange(80., -60., -1. / 112)
    lon_1k = np.arange(-180., 180., 1. / 112)
    lat_300 = ds.lat.values
    lon_300 = ds.lon.values
    ext_1K = np.zeros(4)
    # TODO find a more pythonic way
    ext_1K[0] = find_nearest(lon_1k, my_ext[0]) - 1. / 224
    ext_1K[1] = find_nearest(lat_1k, my_ext[1]) + 1. / 224
    ext_1K[2] = find_nearest(lon_1k, my_ext[2]) - 1. / 224
    ext_1K[3] = find_nearest(lat_1k, my_ext[3]) + 1. / 224
    my_ext[0] = find_nearest(lat_300, ext_1K[0])
    my_ext[1] = find_nearest(lon_300, ext_1K[1])
    my_ext[2] = find_nearest(lat_300, ext_1K[2])
    my_ext[3] = find_nearest(lon_300, ext_1K[3])
    return my_ext

if len(AOI):
    assert AOI[1] >= AOI[3], 'min Latitude is bigger than correspond Max, ' \
                                   'pls change position or check values.'
    assert AOI[0] <= AOI[2], 'min Longitude is bigger than correspond Max, ' \
                                   'pls change position or check values.'
    assert ds.lat[-1] <= AOI[3] <= ds.lat[0], 'min Latitudinal value out of original dataset Max ext.'
    assert ds.lat[-1] <= AOI[1] <= ds.lat[0], 'Max Latitudinal value out of original dataset Max ext.'
    assert ds.lon[0] <= AOI[0] <= ds.lon[-1], 'min Longitudinal value out of original dataset Max ext.'
    assert ds.lon[0] <= AOI[2] <= ds.lon[-1], 'Max Longitudinal value out of original dataset Max ext.'
    adj_ext = bnd_box_adj(AOI)
    da = da.sel(lon=slice(adj_ext[0], adj_ext[2]), lat=slice(adj_ext[1], adj_ext[3]))
else:
    da = da.shift(lat=1, lon=1)

## Step 4: Resampling using the aggregation approach

There are several approaches to resample data to a coarser resolution. 

As both products are provided in a regular latitude/longitude grid (plate carrée), with the ellipsoid WGS 1984 (Terrestrial radius=6378km), a Kernel can be used to achieve the resampling.
Products, usually named with a resolution of 1 km, have a resolution of the grid equal to 1/112°. Instead, products the nominally named os 333 m are defined by a resolution of 1/336°.
According to these values, a grid of 3x3 can be used to aggregate exactly a finner resolution product to the coarsen one.


Most of the products, some specific values are used to define qualitative information. These need to be masked out to avoid bias in the resampling.

Besides, it is advised to include a condition that at least 5 out of the 9 pixels had to have valid values (i.e. not NA). 


In [ ]:
da_msk = da.where(da <= param['DIGITAL_MAX'])

create the coarsen dataset

In [ ]:
coarsen = da_msk.coarsen(lat=3, lon=3, coord_func=np.mean, boundary='trim', keep_attrs=False).mean()

mask the dataset according to the minumum required values

In [ ]:
vo = xr.where(da <= param['DIGITAL_MAX'], 1, 0)
vo_cnt = vo.coarsen(lat=3, lon=3, coord_func=np.mean, boundary='trim', keep_attrs=False).sum()
da_r = coarsen.where(vo_cnt >= 5)

# Step 5: Check the outcome and data consolidation

Data will be consolidated, on the disk, in netCDF format. 

In [ ]:
da_r.name = param['product']
da_r.attrs['short_name'] = param['short_name']
da_r.attrs['long_name'] = param['long_name']
prmts = dict({param['product']: {'dtype': 'f8', 'zlib': 'True', 'complevel': 4}})
name = param['product']
da_r.to_netcdf(rf'D:/Data/CGL_subproject_coarse_res/Tests/CGLS_{name}_1KM_R_Europe_{date}.nc', encoding=prmts)

If needed dataresults can be plotted. 

In [ ]:
da_r.plot(robust=True, cmap='YlGn', figsize=(15, 10))
plt.title(f'Copernicus Global Land\n Resampled {name} to 1K over Europe\n date: {date_h}')
plt.ylabel('latitude')
plt.xlabel('longitude')
plt.draw()
plt.show()